# PR Review Agent - Interactive Demo

This notebook demonstrates how to instantiate and use the PR Review Agent.

## Prerequisites

1. **Environment Setup**: Ensure `agent/.env` is configured with:
   - `GOOGLE_CLOUD_PROJECT` - Your GCP project ID
   - `GOOGLE_CLOUD_LOCATION` - GCP region (e.g., `europe-west2`)
   - `REPOSITORY` - Repo identifier (e.g., `owner/repo`)
   - `PR_NUMBER` - PR/MR number to review
   - `GH_TOKEN` or `GITLAB_TOKEN` - Authentication token

2. **Authentication**:
   - Google Cloud: `gcloud auth application-default login`
   - GitHub: `gh auth login` or set `GH_TOKEN`
   - GitLab: `glab auth login` or set `GITLAB_TOKEN`

3. **Virtual Environment**: Ensure you're running in the project's venv with all dependencies installed

## 1. Import Dependencies and Load Environment

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / 'agent' / '.env')


os.environ['REPOSITORY'] = 'darkin100/i-am-reviewed'
os.environ['PR_NUMBER'] = '5'

print("Environment loaded")
print(f"Project: {os.getenv('GOOGLE_CLOUD_PROJECT')}")
print(f"Location: {os.getenv('GOOGLE_CLOUD_LOCATION')}")
print(f"Repository: {os.getenv('REPOSITORY')}")
print(f"PR/MR Number: {os.getenv('PR_NUMBER')}")

✓ Environment loaded
  Project: iamreleased
  Location: europe-west2
  Repository: None
  PR/MR Number: None


## 2. Import Agent Components

In [ ]:
from google import genai
from agent.platforms import get_platform
from agent.utils import strip_markdown_wrapper

print("✓ Agent components imported")

## 3. Instantiate Platform (GitHub or GitLab)

The agent uses a platform abstraction layer to support multiple Git hosting providers.

In [ ]:
# Choose your platform: 'github' or 'gitlab'
PROVIDER = 'github'  # Change to 'gitlab' if using GitLab

# Get platform implementation
platform = get_platform(PROVIDER)

print(f"✓ Platform instantiated: {platform.get_platform_name()}")
print(f"  Platform class: {type(platform).__name__}")

## 4. Validate Environment Variables

Check that all required variables are set before proceeding.

In [ ]:
# Validate required variables
required_vars = {
    'GOOGLE_CLOUD_PROJECT': os.getenv('GOOGLE_CLOUD_PROJECT'),
    'GOOGLE_CLOUD_LOCATION': os.getenv('GOOGLE_CLOUD_LOCATION'),
    'REPOSITORY': os.getenv('REPOSITORY'),
    'PR_NUMBER': os.getenv('PR_NUMBER')
}

# Platform-specific validation
platform_missing = platform.validate_environment_variables()

missing = [k for k, v in required_vars.items() if not v]
if missing or platform_missing:
    print("❌ Missing required environment variables:")
    for var in missing:
        print(f"  - {var}")
    for var in platform_missing or []:
        print(f"  - {var}")
else:
    print("✓ All environment variables validated")
    
    # Setup platform authentication
    platform.setup_auth()
    print("✓ Platform authentication configured")

## 5. Instantiate Gemini Client

Create the Google GenAI client configured for Vertex AI.

In [ ]:
# Create Gemini client with Vertex AI
client = genai.Client(
    vertexai=True,
    project=os.getenv('GOOGLE_CLOUD_PROJECT'),
    location=os.getenv('GOOGLE_CLOUD_LOCATION')
)

print("✓ Gemini client instantiated")
print(f"  Model: gemini-2.5-flash")
print(f"  Backend: Vertex AI")

## 6. Fetch PR/MR Data

Use the platform abstraction to fetch PR/MR metadata and diff.

In [ ]:
# Get repository and PR number from environment
repo = os.getenv('REPOSITORY')
pr_number = int(os.getenv('PR_NUMBER'))

print(f"Fetching data for {repo} PR/MR #{pr_number}...\n")

# Fetch PR/MR metadata
pr_info = platform.get_pr_info(repo, pr_number)

print("✓ PR/MR Metadata:")
print(f"  Title: {pr_info.get('title', 'N/A')}")
print(f"  Author: {pr_info.get('author', {}).get('login', 'N/A')}")
print(f"  Branch: {pr_info.get('headRefName', 'N/A')} → {pr_info.get('baseRefName', 'N/A')}")
print(f"  State: {pr_info.get('state', 'N/A')}")
print(f"\n  Description: {pr_info.get('body', 'No description')[:100]}...")

In [ ]:
# Fetch PR/MR diff
pr_diff = platform.get_pr_diff(repo, pr_number)

print("✓ PR/MR Diff fetched")
print(f"  Diff size: {len(pr_diff)} characters")
print(f"\n  Preview (first 500 chars):\n{pr_diff[:500]}...")

## 7. Generate AI Review

Send the PR/MR data to Gemini for analysis.

In [ ]:
# Create review prompt
prompt = f"""Review this pull request:

Title: {pr_info.get('title', 'N/A')}

Description:
{pr_info.get('body', 'No description provided')}

Branch: {pr_info.get('headRefName', 'N/A')} -> {pr_info.get('baseRefName', 'N/A')}

Author: {pr_info.get('author', {}).get('login', 'N/A')}

Changes:
{pr_diff}

Provide your code review."""

# System instruction for the AI
system_instruction = """You are a code reviewer analyzing a pull request.

Review the PR for:
- Obvious bugs or logic errors
- Code quality issues (complexity, readability)
- Potential security issues
- Missing error handling
- Best practice violations

Provide a concise review summary with:
1. Overall assessment (Looks good / Needs work / Has issues)
2. Key findings (list 3-5 most important issues)
3. Positive observations (what's done well)

Keep feedback constructive and actionable.
Format your response in markdown for GitHub."""

print("Generating AI review...\n")

# Generate review
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt,
    config=genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.7
    )
)

review_text = response.text

# Clean up any markdown code block wrappers
review_text = strip_markdown_wrapper(review_text)

print("✓ Review generated\n")
print("=" * 80)
print(review_text)
print("=" * 80)

## 8. Post Review Comment (Optional)

Post the generated review as a comment on the PR/MR.

**⚠️ Warning**: This will actually post to the PR/MR. Uncomment to execute.

In [ ]:
# Uncomment the following lines to post the review

# print("Posting review comment...")
# platform.post_pr_comment(repo, pr_number, review_text)
# print(f"✓ Review posted to PR/MR #{pr_number}")

# # Generate view URL
# if platform.get_platform_name() == 'github':
#     print(f"View at: https://github.com/{repo}/pull/{pr_number}")
# elif platform.get_platform_name() == 'gitlab':
#     gitlab_host = os.getenv('CI_SERVER_HOST', 'gitlab.com')
#     print(f"View at: https://{gitlab_host}/{repo}/-/merge_requests/{pr_number}")

print("(Review posting is commented out - uncomment to post)")

## 9. Alternative: Review Without Posting

You can also just generate reviews without posting them, useful for testing.

In [ ]:
# Save review to file instead of posting
output_file = project_root / 'evals' / f'review_pr{pr_number}.md'

with open(output_file, 'w') as f:
    f.write(f"# Review for PR/MR #{pr_number}\n\n")
    f.write(f"**Repository**: {repo}\n")
    f.write(f"**Title**: {pr_info.get('title', 'N/A')}\n\n")
    f.write("---\n\n")
    f.write(review_text)

print(f"✓ Review saved to: {output_file}")

## Summary

This notebook demonstrated:

1. ✅ Loading environment configuration
2. ✅ Instantiating platform abstraction (GitHub/GitLab)
3. ✅ Creating Gemini client with Vertex AI
4. ✅ Fetching PR/MR data through platform API
5. ✅ Generating AI-powered code review
6. ✅ Posting review comments (optional)

### Key Components

- **Platform Abstraction** (`agent.platforms`): Unified interface for GitHub/GitLab
- **Gemini Client** (`google.genai.Client`): AI model interface via Vertex AI
- **Review Generation**: Structured prompts with system instructions

### Next Steps

- Try reviewing different PRs by changing `PR_NUMBER` in `.env`
- Experiment with different system instructions for specialized reviews
- Switch between GitHub and GitLab by changing `PROVIDER`
- Customize the review prompt for specific needs (security, performance, etc.)